# **Cassava ResNet50 fine-tuning**
2021/01/12 written by T.Yonezu

In [12]:
%load_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader, Dataset

import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import glob 
import os
from tqdm import tqdm

from cassava_dataset import *
from myvgg import *

import warnings
warnings.simplefilter('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
input_dir = os.path.join('..',"..", 'input', 'cassava-leaf-disease-classification')

## **Fine-tuning**

In [14]:
x = pd.read_csv(os.path.join(input_dir, 'train.csv'))
x["image_path"] = os.path.join(input_dir,"train_images")
x["image_path"] = x["image_path"].str.cat(x["image_id"], sep=os.path.sep)


train, valid = train_test_split(X, y, test_size=0.33, random_state=42)

train_dict = dict( zip(x["image_path"],x["label"]) )
valid_dict = 

In [15]:
size = (224,224)
mean = [0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

transform = ImageTransform(size,mean,std)

In [16]:
BATCH_SIZE = 10
NUM_WORKERS = 8

train_data = CassavaDataset(train_dict,transform=transform)
train_data = DataLoader(train_data,batch_size=BATCH_SIZE)

train_data = CassavaDataset(train_dict,transform=transform)
train_data = DataLoader(train_data,batch_size=BATCH_SIZE)

In [17]:
EPOCH_NUM = 100

OUT_DIR = os.path.join("..","..","input","cassava-models")
MODEL_NAME = "ResNet50_cassava_trained_%dEpoch.mdl"
PATH = os.path.join(OUT_DIR,MODEL_NAME%EPOCH_NUM)

In [20]:
import torchvision.models as models

model = models.resnet50(pretrained=False)
model.fc = nn.Linear(in_features=2048, out_features=5, bias=True)
model.
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
import torch.optim as optim
from torch import nn

model = model.cuda()
model.train()

optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# in your training loop:
for epoch in tqdm(range(EPOCH_NUM)):
    loss = 0
    
    for batch in (train_data):
        
        X = batch[0].cuda()
        y = batch[1].cuda()
        
        
        pred = model(X)
        
        # zero the gradient buffers
        optimizer.zero_grad()
        loss = criterion(pred, y)
        loss.backward()
        
        loss += loss.item()
        optimizer.step() # Does the update
        
    print("epoch:",epoch,"|loss:",loss)
        

if not(os.path.exists(OUT_DIR)):
    os.makedirs(OUT_DIR)
torch.save(model.state_dict(),PATH)

  0%|                                                                                          | 0/100 [00:09<?, ?it/s]


KeyboardInterrupt: 